In [129]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

#### 处理数据集

<img src="LSTM.png" alt="./" width="500" height="300">

#### 构建LSTM

In [130]:
array = torch.randn([3,5])
array

tensor([[-1.5064,  0.4800, -0.2995, -0.7215,  0.7774],
        [-0.1934,  0.9956,  0.5153, -1.0429, -0.5085],
        [ 0.8412,  0.7855,  0.2901,  0.6171, -1.4021]])

In [131]:
# LSTM Cell
'''
nn.Module :

Base class for all neural network modules.Your models should also subclass this class.
Modules can also contain other Modules, allowing to nest them in a tree structure. You 
can assign the submodules as regular attributes:
'''
class LSTMCell(nn.Module):#LSTM Cell继承自nn.Module

    def __init__(self, input_size, hidden_size):

        " input_size  : Input data size "
        " hidden_size : Hidden state size "
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        combine_size = input_size + hidden_size
        # 定义输入门的线性层
        self.in_gate = nn.Linear(combine_size, hidden_size)
        # 定义遗忘门的线性层
        self.forgot_gate = nn.Linear(combine_size, hidden_size)
        # 定义备选细胞元的线性层
        self.new_cell_state = nn.Linear(combine_size, hidden_size)
        # 定义输出门的线性层
        self.out_gate = nn.Linear(combine_size, hidden_size)

    def forward(self, inputs, state=None):
        '''
        torch.cat是PyTorch中用于连接张量的函数，可以沿指定的维度将多个张量合并为一个张量
        
        向前传播
        参数
        ----
        inputs ：torch.FloatTensor
            输入数据，形状为(B, I)，其中B表示批量大小，I表示文字特征的长度（input_size）
        state ：tuple(torch.FloatTensor, torch.FloatTensor)
            (hidden state，cell state)，两个状态的形状都为(B, H)，其中H表示隐藏状态的长度（hidden_size）
        返回
        ----
        hs ：torch.FloatTensor，hidden state，shape (B, H)
        cs ：torch.FloatTensor，cell state，shape (B, H)
        '''
        B, _ = inputs.shape
        if state is None:
            state = self.init_state(B, inputs.device)
        hs, cs = state
        combined = torch.cat((inputs, hs), dim=1)           # (B, I + H)
        # 输入门
        ingate = F.sigmoid(self.in_gate(combined))          # (B,     H)
        # 遗忘门
        forgetgate = F.sigmoid(self.forgot_gate(combined))  # (B,     H)
        # 输出门
        outgate = F.sigmoid(self.out_gate(combined))        # (B,     H)
        # 更新细胞状态
        ncs = F.tanh(self.new_cell_state(combined))         # (B,     H)
        cs = (forgetgate * cs) + (ingate * ncs)             # (B,     H)
        # 更新隐藏状态
        hs = outgate * F.tanh(cs)                           # (B,     H)
        return hs, cs
    
    def init_state(self, B, device):
        # 默认的隐藏状态和细胞状态全部都等于0
        cs = torch.zeros((B, self.hidden_size), device=device) # Cell state
        hs = torch.zeros((B, self.hidden_size), device=device) # Hidden state
        return hs, cs


In [132]:
# LSTM Network
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size):
        '''
        单层的长短期记忆网络（支持批量计算）
        参数
        ----
        input_size ：int，输入数据的特征长度
        hidden_size ：int，隐藏状态的特征长度
        '''
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm = LSTMCell(self.input_size, self.hidden_size)

    def forward(self, inputs, state=None):
        '''
        向前传播
        参数
        ----
        inputs ：torch.FloatTensor
            输入数据的集合，形状为(B, T, C)，其中B表示批量大小，T表示文本长度，C表示文字特征的长度（input_size）
        state ：tuple(torch.FloatTensor, torch.FloatTensor)
            (初始的隐藏状态，初始的细胞状态)，两个状态的形状都为(B, H)，其中H表示隐藏状态的长度（hidden_size）
        返回
        ----
        hidden ：torch.FloatTensor，所有隐藏状态的集合，形状为(B, T, H)
        '''
        re = []
        B, T, C = inputs.shape
        inputs = inputs.transpose(0, 1)  # (T, B, C)
        for i in range(T):
            state = self.lstm(inputs[i], state)
            # 只记录隐藏状态，state[0]的形状为(B, H)
            re.append(state[0])
        result_tensor = torch.stack(re, dim=0)  # (T, B, H)
        return result_tensor.transpose(0, 1)    # (B, T, H)

In [133]:
B, T, input_size, hidden_size, num_layers = torch.randint(1, 20, (5,)).tolist()
B

12

In [134]:
#.tolist()：将生成的张量转换为 Python 的列表
print(torch.randint(1,10,(5,5)))
print(torch.randint(1,10,(5,5)).tolist())
print(torch.randint(1, 20, (5,)).tolist()) #.tolist()：将生成的张量转换为 Python 的列表)

tensor([[2, 9, 8, 2, 1],
        [2, 1, 3, 3, 1],
        [8, 8, 8, 4, 9],
        [2, 4, 7, 6, 5],
        [8, 8, 7, 5, 2]])
[[4, 7, 1, 2, 7], [2, 5, 6, 9, 7], [2, 8, 9, 3, 2], [9, 6, 8, 2, 7], [7, 4, 3, 8, 8]]
[11, 8, 18, 2, 11]


In [135]:
def test_lstm():
    '''
    测试LSTM实现的准确性
    '''
    # 随机生成模型结构
    B, T, input_size, hidden_size, num_layers = torch.randint(1, 20, (5,)).tolist() #.tolist()：将生成的张量转换为 Python 的列表
    ref_model = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
    # 随机生成输入
    inputs = torch.randn(B, T, input_size)
    hs, cs = torch.randn((2 * num_layers, B, hidden_size)).chunk(2, 0)
    re = inputs
    # 取出模型参数
    for layer_index in range(num_layers):
        l = ref_model.all_weights[layer_index]
        if layer_index == 0:
            model = LSTM(input_size, hidden_size)
        else:
            model = LSTM(hidden_size, hidden_size)
        i, f, c, o = torch.cat((l[0], l[1]), dim=1).chunk(4, 0)
        ib, fb, cb, ob = (l[2] + l[3]).chunk(4, 0)
        # 设置模型参数
        model.lstm.in_gate.weight = nn.Parameter(i)
        model.lstm.in_gate.bias = nn.Parameter(ib)
        model.lstm.forgot_gate.weight = nn.Parameter(f)
        model.lstm.forgot_gate.bias = nn.Parameter(fb)
        model.lstm.new_cell_state.weight = nn.Parameter(c)
        model.lstm.new_cell_state.bias = nn.Parameter(cb)
        model.lstm.out_gate.weight = nn.Parameter(o)
        model.lstm.out_gate.bias = nn.Parameter(ob)
        # 计算隐藏状态
        re = model(re, (hs[layer_index], cs[layer_index]))
    ref_re, _ = ref_model(inputs, (hs, cs))
    # 验证计算结果（最后一层的隐藏状态是否一致）
    out = torch.all(torch.abs(re - ref_re) < 1e-4)
    return out, (B, T, input_size, hidden_size, num_layers)

In [136]:
test_lstm()

(tensor(True), (12, 16, 3, 8, 7))

#### 定义一些参数

In [137]:
batch_size = 64  # Batch size for training.
epochs = 1000  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 12000  # Number of samples to train on.
data_path = '../cmn-eng/cmn.txt'

#### 数据集处理

In [138]:
# python3中，读取文件有三种方法：read（）、readline（）、readlines（）
with open(data_path, 'r', encoding='utf-8') as f:
    T = f.readlines()
T

['Hi.\t嗨。\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #891077 (Martha)\n',
 'Hi.\t你好。\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4857568 (musclegirlxyp)\n',
 'Run.\t你用跑的。\tCC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #3748344 (egg0073)\n',
 'Stop!\t住手！\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (CM) & #448321 (GlossaMatik)\n',
 'Wait!\t等等！\tCC-BY 2.0 (France) Attribution: tatoeba.org #1744314 (belgavox) & #4970122 (wzhd)\n',
 'Wait!\t等一下！\tCC-BY 2.0 (France) Attribution: tatoeba.org #1744314 (belgavox) & #5092613 (mirrorvan)\n',
 'Begin.\t开始！\tCC-BY 2.0 (France) Attribution: tatoeba.org #6102432 (mailohilohi) & #5094852 (Jin_Dehong)\n',
 'Hello!\t你好。\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #4857568 (musclegirlxyp)\n',
 'I try.\t我试试。\tCC-BY 2.0 (France) Attribution: tatoeba.org #20776 (CK) & #8870261 (will66)\n',
 'I won!\t我赢了。\tCC-BY 2.0 (France) Attribution: tatoeba.org #2005192 (CK) & #5102367 (mirr

In [139]:
# Vectorize the data.
lines = []
input_texts = [] # 保存英文数据集
target_texts = [] # 保存中文数据集
input_characters = set() # 保存英文字符，比如a,b，c
target_characters = set() # 保存中文字符,比如，你，我，她
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')# 一行一行读取数据
for line in lines[: min(num_samples, len(lines) - 1)]: # 遍历每一行数据集（用min来防止越出)
    input_text, target_text, _ = line.split('\t')
    input_texts.append(input_text)
    target_texts.append(target_text)
    # 提取字符
    for char in input_text: 
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [140]:
print(input_texts)
print(target_texts)
print(input_characters)
print(target_characters)

['Hi.', 'Hi.', 'Run.', 'Stop!', 'Wait!', 'Wait!', 'Begin.', 'Hello!', 'I try.', 'I won!', 'Oh no!', 'Cheers!', 'Got it?', 'Got it?', 'Got it?', 'He ran.', 'Hop in.', 'I know.', 'I quit.', 'I quit.', "I'm OK.", "I'm up.", 'Listen.', 'No way!', 'No way!', 'Really?', 'Really?', 'Thanks!', 'Try it.', 'We try.', 'Why me?', 'Ask Tom.', 'Awesome!', 'Be calm.', 'Be fair.', 'Be kind.', 'Be kind.', 'Be nice.', 'Be nice.', 'Call me.', 'Call us.', 'Come in.', 'Get Tom.', 'Get out!', 'Get out!', 'Go away!', 'Go away!', 'Go away.', 'Go home.', 'Go home.', 'Goodbye!', 'Goodbye!', 'Hang on!', 'Hang on!', 'Hang on.', 'He came.', 'He runs.', 'Help me.', 'Help us.', 'Hit Tom.', 'Hold on.', 'Hug Tom.', 'Hug Tom.', 'I agree.', "I'm hot.", "I'm ill.", "I'm sad.", "I'm shy.", "I'm wet.", "It's OK.", "It's me.", 'Join us.', 'Keep it.', 'Kiss me.', 'Perfect!', 'See you.', 'Shut up!', 'Skip it.', 'Take it.', 'Tell me.', 'Tom won.', 'Wake up!', 'Wash up.', 'We know.', 'Welcome.', 'Who won?', 'Why not?', 'You run

In [141]:
input_characters = sorted(list(input_characters)) # 排序一下
target_characters = sorted(list(target_characters))

In [142]:
print(input_characters)
print(target_characters)

[' ', '!', '"', '$', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ō', '’']
[' ', '!', '"', '(', ')', ',', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'F', 'J', 'M', 'O', 'P', 'T', 'U', 'W', 'a', 'c', 'e', 'f', 'h', 'i', 'm', 'o', 'r', 's', 't', 'w', 'y', '\u200b', '“', '”', '、', '。', '一', '丁', '七', '万', '丈', '三', '上', '下', '不', '与', '丐', '专', '世', '丘', '业', '丛', '东', '丝', '丟', '丢', '两', '严', '並', '丧', '个', '中', '临', '为', '主', '丽', '举', '久', '么', '义', '之', '乎', '乏', '乐', '乘', '九', '乞', '也', '习', '乡', '书', '买', '乱', '乳', '乾', '亂', '了', '予', '争', '事', '二', '于', '云', '互', '五', '井', '亚', '些', '亡', '交', '产', '享', '京', '亮', '亲', '人', '什', '仁', '仅', '仇', '今'

In [143]:
num_encoder_tokens = len(input_characters) # 英文字符数量
num_decoder_tokens = len(target_characters) # 中文文字数量
max_encoder_seq_length = max([len(txt) for txt in input_texts]) # 输入的最长句子长度
max_decoder_seq_length = max([len(txt) for txt in target_texts])# 输出的最长句子长度

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 12000
Number of unique input tokens: 74
Number of unique output tokens: 2628
Max sequence length for inputs: 27
Max sequence length for outputs: 20


In [144]:
 # mapping token to index， easily to vectors
# 处理方便进行编码为向量
# {
#   'a': 0,
#   'b': 1,
#   'c': 2,
#   ...
#   'z': 25
# }
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, 'Z': 45, 'a': 46, 'b': 47, 'c': 48, 'd': 49, 'e': 50, 'f': 51, 'g': 52, 'h': 53, 'i': 54, 'j': 55, 'k': 56, 'l': 57, 'm': 58, 'n': 59, 'o': 60, 'p': 61, 'q': 62, 'r': 63, 's': 64, 't': 65, 'u': 66, 'v': 67, 'w': 68, 'x': 69, 'y': 70, 'z': 71, 'ō': 72, '’': 73}
{' ': 0, '!': 1, '"': 2, '(': 3, ')': 4, ',': 5, '.': 6, '/': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'F': 24, 'J': 25, 'M': 26, 'O': 27, 'P': 28, 'T': 29, 'U': 30, 'W': 31, 'a': 32, 'c': 33, 'e': 34, 'f': 35, 'h': 36, 'i': 37, 'm': 38, '

In [145]:
# np.zeros(shape, dtype, order)
# shape is an tuple, in here 3D
encoder_input_data = np.zeros( # (12000, 32, 73) （数据集长度、句子长度、字符数量）
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros( # (12000, 22, 2751)
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros( # (12000, 22, 2751)
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print("形状为(B, T, C)，其中B表示批量大小，T表示文本长度，C表示文字特征的长度（input_size）")
print("(B,T,C)", encoder_input_data.shape)
print("(B,T,C)", decoder_input_data.shape)
print("(B,T,C)", decoder_target_data.shape)

形状为(B, T, C)，其中B表示批量大小，T表示文本长度，C表示文字特征的长度（input_size）
(B,T,C) (12000, 27, 74)
(B,T,C) (12000, 20, 2628)
(B,T,C) (12000, 20, 2628)


#### 编码数据

In [159]:
a = ['a', 'b', 'c', 'd']
b = ['1', '2', '3', '4']
list(zip(a, b))#函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组
[('a', '1'), ('b', '2'), ('c', '3'), ('d', '4')]

[('a', '1'), ('b', '2'), ('c', '3'), ('d', '4')]

In [172]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        # 3D vector only z-index has char its value equals 1.0
        encoder_input_data[i, t, input_token_index[char]] = 1.

    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            # igone t=0 and start t=1, means
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

#### 训练模型